# 37009 Workshop Week 4: Risk Factor Mapping

Below are the solutions to the exercises for the Week 4 workshop. Steps indicated by **ASSUMPTION** are modelling assumptions and can be varied when implementing these codes. Therefore, I recommend paying attention to the work flow presented below and, as an exercise, modify the codes to cater to different modelling assumptions.

**ASSUMPTION:** A day-count convention of Actual/360 is assumed throughout the exercise.\

**ASSUMPTION:** All risk factor mappings are with respect to the one-day percentage returns in the risk factors.

*REMARK:* A discussion of log-returns and its use in risk factor mapping can be found in the Week 4 Workshop notes (Section 3.5). In summary, the use of either one-day log-returns or percentage returns is a modelling assumption, but one must recognize that the time-scaling property of VaR, which we do not use at this point, follows primarily from the use of log-returns. **As an exercise, you can re-do all calculations below using log-returns instead of percentage returns and see the difference, especially in the calculated correlation and covariance matrices.**

In [1]:
# Required libraries
import numpy as np
import scipy as sp
import scipy.stats
import pandas as pd
import math
from datetime import datetime
from datetime import timedelta

*REMARK:* Some variable names may be repeated throughout the entire notebook.

## Cash Flow Mapping

The codes below automate the two-vertex duration- or volatility-invariant cash flow mapping. The PV-invariance constraint is implicity implosed. The function allows the user to choose the left- and right-maturities for the cash flow mapping (which is a modelling assumption). If the user requests a volatility-invariant mapping, the user has to provide the standard deviations and correlations of the **zero coupon bond returns**.

Exercise (1) will not be answered here as it consists of plugging in the provided values into the function below.

In [39]:
# Cash flow mapping using two vertices
def CFmap2V(maturity, left_maturity, right_maturity, map_type = "Duration", 
            left_vol = None, right_vol = None, left_right_cor = None):
    
    # Assumption: Cash flow at non-standard maturity is mapped to two vertices, one on the left and one on the right.
    # Imposes PV-invariance; may impose duration-invariance or volatility-invariance, but not both.
    
    # Output: alpha parameter to allocate a PV of 1 to the left vertex T1. 
    # For the volatility mapping, the function returns two values of alpha. 
    # We often will choose the alpha that is between 0 and 1.
    
    # Use mathematical notation for function inputs
    T = maturity
    T1 = left_maturity
    T2 = right_maturity
    
    if map_type == "Duration":
        alpha = (T2 - T) / (T2 - T1)
        return alpha
    
    if map_type == "Volatility":
        
        if (left_vol == None or right_vol == None or left_right_cor == None):
            print("Please provide non-empty volatility-invariant mapping inputs")
        
        else:
            # Extract volatility-invariant CF map inputs
            sigma1 = left_vol
            sigma2 = right_vol
            rho = left_right_cor
            
            # Linearly interpolate the variance at the non-standard maturity
            sq_sigma_interp = np.interp(x = T, xp = np.array([T1, T2]), fp = np.array([sigma1 ** 2, sigma2 ** 2]))
            
            # Coefficients of quadratic equation for alpha
            a_coeff = sigma1 ** 2 - 2 * rho * sigma1 * sigma2 + sigma2 ** 2
            b_coeff = 2 * rho * sigma1 * sigma2 - 2 * sigma2 ** 2
            c_coeff = sigma2 ** 2 - sq_sigma_interp
            
            # Solve quadratic equation for alpha
            alpha1 = (-b_coeff + np.sqrt(b_coeff ** 2 - 4 * a_coeff * c_coeff)) / (2 * a_coeff)
            alpha2 = (-b_coeff - np.sqrt(b_coeff ** 2 - 4 * a_coeff * c_coeff)) / (2 * a_coeff)
            alpha = np.array([alpha1, alpha2])
            
            return alpha

## CF Mapping for Bonds

The codes below implement the cash flow mapping required in Exercise 2(b) of the Week 4 Workshop Exercises. Exercise 2(a) is answer as part of the volatility-invariant mapping chosen for this implementation.

In [2]:
# Determine dates of future cash flows for the bond
date_today = datetime(2022, 6, 30)
date_today_str = '2022-06-30' # For pulling values out of the given market data, as they are (row-)indexed by date
coupon_rate = 0.0475
coupon_freq = 2
principal = 100

payment_dates = np.array([datetime(2022, 11, 30), datetime(2023, 5, 30), datetime(2023, 11, 30), 
                          datetime(2024, 5, 30), datetime(2024, 11, 30)])
payment_dates_from_today = payment_dates - date_today

cash_flows = np.array([principal * coupon_rate / coupon_freq] * len(payment_dates))
cash_flows[len(payment_dates) - 1] = cash_flows[len(payment_dates) - 1] + principal

bond_summary = pd.DataFrame({'Time':payment_dates_from_today, 'CF': cash_flows})
bond_summary['Time'] = bond_summary['Time'].dt.days / 360
print(bond_summary)

       Time       CF
0  0.425000    2.375
1  0.927778    2.375
2  1.438889    2.375
3  1.944444    2.375
4  2.455556  102.375


At this point, we decide the standard vertices (O/N, 1 month, 3 months, 6 months, 9 months, 1 year, 2 years, 3 years) to which we map each of the future cash flows.

**ASSUMPTION:** Map each cash flow to the standard vertices immediately to the left and to the right of the time when the cash flow is due.

Next, we decide which cash flow mapping technique to use. For illustrative purposes, we shall use the PV-invariant and volatility-invariant mapping. 

**ASSUMPTION:** Use PV-invariant and volatility-invariant cash flow mapping to map the bond's cash flows to standard maturities.

This requires us to calculate the covariances and correlations of returns of the zero coupond bond prices at the standard maturities.

*REMARK:* The duration-invariant mapping is easier to implement as it only requires the time points as inputs for the mapping. This shall be left as an exercise for you.

In [3]:
# Compute one-day percentage returns in zero coupon bond prices
auszero = pd.read_csv('AustraliaZeroCurve.csv', index_col = 0)
maturities = np.array([1/360, 3/12, 6/12, 9/12, 1, 2, 3, 4, 5, 6, 7])
auszerobond = np.exp(- auszero * maturities / 100)     # Divide by 100 since the zero rates are in percentage POINTS
auszerobond_ret = auszerobond.pct_change().tail(-1)

# Compute covariance and correlation matrices in ZCB price returns
auszerobond_ret_cov = auszerobond_ret.cov()
auszerobond_ret_corr = auszerobond_ret.corr()

We are now ready to perform the volatility-invariant cash flow mapping. The following codes automate the process using a for-loop over all maturities. The for-loop may also go through future cash flows that occur at a standard maturity, but will perform a check before implementing the volatility-invariant mapping via `CFmap2V()`.

Before the for-loop, we construct an empty data frame `cf_summary` (containing only zeros) with number of rows equal to the number of future cash flows and number of columns equal to the number of standard maturities. This is where we will store the cash flow mapping results.

*REMARK:* Commented out below is an attempt to summarize the cash flows using the `bond_summary` table generated above. However, it is difficult to obtain the **total** cash flow mapped to each standard vertex using this layout.

In [1]:
# Cash flow mapping for all future cash flows

# # Construct blank columns in bond_summary to store the cash flow mapping results
# bond_summary['LeftVertex'] = bond_summary['RightVertex'] = bond_summary['MapCoeff'] = None

# Alternative summary of cash flow mapping results
cf_summary = pd.DataFrame(0, index = bond_summary['Time'], columns = maturities)

for i in range(0, len(payment_dates)):
    
    # Extract maturity of ith cash flow
    nonstd_maturity = bond_summary['Time'].loc[i]
    
    # Determine the left and right standard vertices
    left_time_subset = np.where(maturities <= nonstd_maturity)
    left_mat_ind = max(left_time_subset[0])
    left_mat = maturities[left_mat_ind]
    
    right_time_subset = np.where(maturities >= nonstd_maturity)
    right_mat_ind = min(right_time_subset[0])
    right_mat = maturities[right_mat_ind]
    
    # Check if maturity of the ith cash flow is a standard maturity
    if (nonstd_maturity == left_mat) or (nonstd_maturity == right_mat):
        map_coeff = 1
        
    else: # Proceed with cash flow mapping
        
        # Extract volatilities and correlation from ZCB return covariance and correlation matrices
        right_mat_var = auszerobond_ret_cov.iloc[right_mat_ind, right_mat_ind]
        left_mat_var = auszerobond_ret_cov.iloc[left_mat_ind, left_mat_ind]
        left_right_corr = auszerobond_ret_corr.iloc[right_mat_ind, left_mat_ind]
        
        # Implement volatility-invariant cash flow mapping
        map_coeff = CFmap2V(maturity = nonstd_maturity, left_maturity = left_mat, right_maturity = right_mat,
                            map_type = "Volatility", left_vol = left_mat_var ** 0.5, right_vol = right_mat_var ** 0.5,
                            left_right_cor = left_right_corr)
        map_coeff = map_coeff[np.where((map_coeff >= 0) & (map_coeff <= 1))[0]]
    
#     # Summarize results
#     bond_summary.at[i, 'LeftVertex'] = left_mat
#     bond_summary.at[i, 'RightVertex'] = right_mat
#     bond_summary.at[i, 'MapCoeff'] = map_coeff[0]
    
    # Cash flow mapping summary
    cf_summary.at[nonstd_maturity, left_mat] = map_coeff[0]
    cf_summary.at[nonstd_maturity, right_mat] = 1 - map_coeff[0]

# bond_summary['CFatLeftVertex'] = bond_summary['CF'] * bond_summary['MapCoeff']
# bond_summary['CFatRightVertex'] = bond_summary['CF'] * (1 - bond_summary['MapCoeff'])

NameError: name 'pd' is not defined

In [5]:
# Multiply each column of cf_summary by the cash flows in bond_summary
cf_summary = cf_summary.apply(lambda x: np.asarray(x) * np.asarray(bond_summary['CF']))

# Obtain column sums to determine the amount mapped to each standard vertex
cf_summary = cf_summary.sum()
cf_summary

0.002778     0.000000
0.250000     0.504374
0.500000     1.870626
0.750000     0.555421
1.000000     2.896767
2.000000    51.186511
3.000000    54.861301
4.000000     0.000000
5.000000     0.000000
6.000000     0.000000
7.000000     0.000000
dtype: float64

The value of the bond therefore has the following representation with respect to zero coupon bonds expiring at the standard maturities, 

\begin{align*}
    B & = 0.5050 \cdot P(0,0.25) + 1.8700 \cdot P(0,0.5) + 0.5538 \cdot P(0,0.75) + 2.8923 \cdot P(0,1)\\
      & \qquad + 50.8841 \cdot P(0,2) + 55.1698 \cdot P(0,3)
\end{align*}

However, for VaR calculations, we need to map the *change in* the bond value to the percentage returns in the risk factors,

\begin{align*}
    \Delta B
        & = 0.5050 \cdot \Delta P(0,0.25) + 1.8700 \cdot \Delta P(0,0.5) + 0.5538 \cdot \Delta P(0,0.75) + 2.8923 \cdot \Delta P(0,1)\\
        & \qquad + 50.8841 \cdot \Delta P(0,2) + 55.1698 \cdot \Delta P(0,3)\\
        & = 0.5050  P(0,0.25) \frac{\Delta P(0,0.25)}{P(0,0.25)} + 1.8700 P(0,0.5) \frac{\Delta P(0,0.5)}{P(0,0.5)} + 0.5538 P(0,0.75) \frac{\Delta P(0,0.75)}{P(0,0.75)} + 2.8923 P(0,1) \frac{\Delta P(0,1)}{P(0,1)}\\
        & \qquad + 50.8841 P(0,2) \frac{\Delta P(0,2)}{P(0,2)} + 55.1698 P(0,3) \frac{\Delta P(0,3)}{P(0,3)}.
\end{align*}

The coefficient of each $P(0,T_i)$ is called the *(dollar) exposure* of the bond value to the risk factor $\frac{\Delta P(0,T_i)}{P(0,T_i)}$ and is given by the present value of the cash flow mapped to that standard vertex (e.g. $0.5050 P(0,0.25)$ is the present value of the cash flow received at the maturity $T = 0.25$).

The codes below calculate the present value of each of the mapped cash flows. 

In [6]:
# Extract discount factors/zero coupon bond prices for today (convert to array)
discount_factors = auszerobond.loc[date_today_str].to_numpy()

# Compute discounted cash flows and summarize in tabular form
cf_summary_df = pd.DataFrame({'CF' : cf_summary, 'DF' : discount_factors})
cf_summary_df['PV of CF'] = cf_summary_df['CF'] * cf_summary_df['DF']
cf_summary_df

,CF,DF,PV of CF
0.002778,0.000000,0.999978,0.000000
0.250000,0.504374,0.995933,0.502323
0.500000,1.870626,0.988862,1.849792
0.750000,0.555421,0.980542,0.544614
1.000000,2.896767,0.972097,2.815937
2.000000,51.186511,0.941953,48.215282
3.000000,54.861301,0.912014,50.034272
4.000000,0.000000,0.879502,0.000000
5.000000,0.000000,0.846200,0.000000
6.000000,0.000000,0.813020,0.000000


The `PV of CF` column in the table above provides the dollar exposure of the bond value to each risk factor. This completes the risk factor mapping for the bond (using volatility-invariant cash flow mapping).

## Risk Factor Mapping for Currency Forward Contracts

Recall that the value of a currency forward today ($t = 0$) is given by 

$$f := f(0) = S(0) e^{-r_f T} - K e^{-r T} = s P_f(0,T) - K P_d(0,T),$$ 

where $s := S(0)$ is the spot exchange rate today (domestic currency per unit foreign currency), $K$ is the contractual forward exchange rate (domestic currency per unit foreign currency), $P_f(0,T)$ is the price of a zero coupon bond in the foreign market with maturity $T$ (corresponding to the foreign zero rate $r_f$), and $P_d(0,T)$ is the price of a zero coupon bond in the domestic market with maturity $T$ (corresponding to the domestic zero rate $r$).

To calculate the value of the contract, we need data on AUD-USD exchange rates `AUDExchangeRates.csv`, Australian zero rates (already loaded above), and US zero rates `USZeroCurve.csv`.

In [21]:
# Load data files
audfxrate = pd.read_csv('AUDExchangeRates.csv', index_col = 0)
uszero = pd.read_csv('USZeroCurve.csv', index_col = 0)

# Express exchange rates in domestic currency per unit foreign currency
audfxrate = 1 / audfxrate

# Compute US zero coupon bond prices
maturities = np.array([1/360, 3/12, 6/12, 9/12, 1, 2, 3, 4, 5, 6, 7])
uszerobond = np.exp(-uszero * maturities / 100)

# Extract data required for mark-to-market valuation
fwd_mat =  3 / 12
fwd_fx = 1 / 0.75
spot_fx = audfxrate.at['2021-12-21', 'USD']
zcb_f = uszerobond.at['2021-12-21', 'US00Y03']
zcb_d = auszerobond.at['2021-12-21', 'AU00Y03']

# Calculate foward contract value (on USD 1.00)
fwd_val = spot_fx * zcb_f - fwd_fx * zcb_d     # Value in AUD per USD
fwd_val

0.014286231683982242

In what follows, it is sufficient to map the change in the forward contract value written on USD 1.00, since the change scales linearly in the number of units of the underlying. The value of the entire position will be `fwd_val * 100,000,000`.

**ASSUMPTION:** We assume that changes in the forward contract are linear in the returns of the risk factors.

From page 7 of the Week 4 Workshop notes, the (linear) mapping of changes in the forward contract value to returns of the spot exchange rate, the foreign zero *rate*, and the domestic zero *rate* is given by

$$\Delta f = se^{-r_f T} \frac{\Delta s}{s} - s r_f T e^{-r_f T} \frac{\Delta r_f}{r_f} + K r T e^{-rT}\frac{\Delta r}{r} = \left[\begin{array}{ccc} se^{-r_f T} & - s r_f T e^{-r_f T} & K r T e^{-rT}\end{array}\right] \left[\begin{array}{c} \frac{\Delta s}{s} \\ \frac{\Delta r_f}{r_f} \\ \frac{\Delta r}{r}\end{array}\right].$$

*REMARK:* In this case, $\Delta f$ represents the change in the value of a currency forward contract written on USD 1.00. To get the change in the value of the forward contract on USD 100 million, we multiply $\Delta f$ by 100,000,000.

We first (manually) compute the exposure of the forward contract to each risk factor. For this, we also need to know the appropriate zero rates in the foreign and domestic markets.

In [22]:
r_f = uszero.at['2021-12-21', 'US00Y03']
r_d = auszero.at['2021-12-21', 'AU00Y03']

exposure = np.array([spot_fx * zcb_f, -spot_fx * r_f * fwd_mat * zcb_f, fwd_fx * r_d * fwd_mat * zcb_d])
exposure

array([ 1.34751957, -0.95454917,  0.00999925])

To complete the risk factor mapping, we construct the covariance and correlation matrices of the risk factors.

In [36]:
# Construct time series of risk factors from given data
risk_factors = pd.DataFrame({'SpotFx' : audfxrate['USD'], 'USZero' : uszero['US00Y03'], 'AUSZero' : auszero['AU00Y03']})

# Construct one-day percentage returns of the risk factors
risk_factors_ret = risk_factors.pct_change().tail(-1)

# Determine covariance and correlation matrices
risk_factors_cov = risk_factors_ret.cov()
risk_factors_corr = risk_factors_ret.corr()

print(risk_factors_cov)
print(risk_factors_corr)

               SpotFx        USZero   AUSZero
SpotFx   3.852277e-05  4.537494e-07 -0.000022
USZero   4.537494e-07  5.839251e-05 -0.000017
AUSZero -2.153800e-05 -1.722416e-05  0.020375
           SpotFx    USZero   AUSZero
SpotFx   1.000000  0.009567 -0.024236
USZero   0.009567  1.000000 -0.015576
AUSZero -0.024236 -0.015576  1.000000


An alternative mapping is with respect to the spot exchange rate, the foreign zero *coupon bond*, and the domestic zero *coupon bond* (see equation (5) of page 8 of the notes),

$$\Delta f = s P_f \frac{\Delta s}{s} + s P_f \frac{\Delta P_f}{P_f} - K P_d \frac{\Delta P_d}{P_d} = \left[\begin{array}{ccc} s P_f & s P_f & -K P_d\end{array}\right] \left[\begin{array}{c} \frac{\Delta s}{s} \\ \frac{\Delta P_f}{P_f} \\ \frac{\Delta P_d}{P_d} \end{array}\right].$$

(We omit the time arguments in the notation for the zero coupon bond prices as we do not consider any other maturities/expiry dates in this problem.) The exposures with respect to each risk factor and the covariance and correlation matrices of the risk factors are calculated below.

In [37]:
# Calculate exposures
exposure2 = np.array([spot_fx * zcb_f, spot_fx * zcb_f, -fwd_fx * zcb_d])
print(exposure2)

# Construct time series of risk factors from given data
risk_factors2 = pd.DataFrame({'SpotFx' : audfxrate['USD'], 'USZeroBond' : uszerobond['US00Y03'], 
                              'AUSZeroBond' : auszerobond['AU00Y03']})

# Construct one-day percentage returns of the risk factors
risk_factors2_ret = risk_factors2.pct_change().tail(-1)

# Determine covariance and correlation matrices
risk_factors2_cov = risk_factors2_ret.cov()
risk_factors2_corr = risk_factors2_ret.corr()

print(risk_factors2_cov)
print(risk_factors2_corr)

[ 1.34751957  1.34751957 -1.33323334]
                   SpotFx    USZeroBond   AUSZeroBond
SpotFx       3.852277e-05 -2.991834e-09 -6.075215e-09
USZeroBond  -2.991834e-09  1.821241e-09 -1.746161e-10
AUSZeroBond -6.075215e-09 -1.746161e-10  2.602657e-09
               SpotFx  USZeroBond  AUSZeroBond
SpotFx       1.000000   -0.011295    -0.019186
USZeroBond  -0.011295    1.000000    -0.080203
AUSZeroBond -0.019186   -0.080203     1.000000


## Risk Factor Mapping for European Options

Before we perform the risk factor mapping, we first recall the formulas/codes for the European option price, delta, and gamma under the Black-Scholes-Merton model.

In [40]:
# Pricing function for European calls and puts
def BSprice(spot_price, strike_price, time_to_maturity, risk_free_rate, dividend_yield, volatility, option_type):
    
    # Use mathematical notation for function inputs
    s = spot_price
    K = strike_price
    tau = time_to_maturity # (T-t) in notes
    r = risk_free_rate
    q = dividend_yield
    sigma = volatility
    
    # Calculate d1 and d2
    d1 = (np.log(s / K) + (r - q + 0.5 * sigma ** 2) * tau) / (sigma * np.sqrt(tau))
    d2 = d1 - sigma * np.sqrt(tau)
    
    # Calculate option prices
    if option_type == 'call':
        price = (np.exp(-q * tau) * s * sp.stats.norm.cdf(d1, 0.0, 1.0) \
                 - np.exp(-r * tau) * K * sp.stats.norm.cdf(d2, 0.0, 1.0))
        
    if option_type == 'put':
        price = (np.exp(-r * tau) * K * sp.stats.norm.cdf(-d2, 0.0, 1.0) \
                 - np.exp(-q * tau) * s * sp.stats.norm.cdf(-d1, 0.0, 1.0))
        
    return price

In [41]:
# Function to calculate option delta under the Black-Scholes-Merton model
def BSdelta(spot_price, strike_price, time_to_maturity, risk_free_rate, dividend_yield, volatility, option_type):
    
    # Use mathematical notation for function inputs
    s = spot_price
    K = strike_price
    tau = time_to_maturity # (T-t) in notes
    r = risk_free_rate
    q = dividend_yield
    sigma = volatility
    
    # Calculate d1 and d2
    d1 = (np.log(s / K) + (r - q + 0.5 * sigma ** 2) * tau) / (sigma * np.sqrt(tau))
    
    # Calculate option delta
    if option_type == 'call':
        value = sp.stats.norm.cdf(d1, 0.0, 1.0)
    
    if option_type == 'put':
        value = sp.stats.norm.cdf(d1, 0.0, 1.0) - 1
    
    return value    

In [42]:
# Function to calculate option gamma under the Black-Scholes-Merton model
def BSgamma(spot_price, strike_price, time_to_maturity, risk_free_rate, dividend_yield, volatility):
    
    # Use mathematical notation for function inputs
    s = spot_price
    K = strike_price
    tau = time_to_maturity # (T-t) in notes
    r = risk_free_rate
    q = dividend_yield
    sigma = volatility
    
    # Compute option gamma (same for calls and puts)
    d1 = (np.log(s / K) + (r - q + 0.5 * sigma ** 2) * tau) / (sigma * np.sqrt(tau))
    value = sp.stats.norm.pdf(d1, 0.0, 1.0) / (s * sigma * np.sqrt(tau))
    
    return value

To calculate the required quantities, we need the following inputs:
1. Spot price of one share of stock of CBA $s$ (get from data)
2. Strike price $K = 95$
3. Time to maturity $T = 6/12 = 0.5$
4. (Australian) risk-free rate with 6-month maturity $r$ (get from data)
5. ANNUALIZED volatility of returns in the stock price $\sigma$ ()

An estimator for $\sigma$ is given by 

$$\hat{\sigma} = \frac{\text{Std. Dev. of daily LOG-returns of the stock price}}{\sqrt{1 / 360}}$$

To compute this estimator, we will use the entire time series of CBA stock prices.

*REMARK:* The definition of $\sigma$ as the annualized volatility of the underlying stock price and the consequent estimator is discussed in further detail in e.g. Hull (2021; Chapter 15). Essentially, this results from the Black-Scholes-Merton model assumption that the log-returns have a normal distribution (i.e. stock prices have a log-normal distribution)

*REMARK:* The denominator depends on the chosen day-count convention (specifically the equivalent of one day in years).

In [57]:
# Load stock price data
stockprice = pd.read_csv('AustraliaStockPrices.csv', index_col = 0)
stockprice.head()

# Set inputs
spot = stockprice.at['2022-07-29', 'CBA']
strike = 95
maturity = 0.5
r = auszero.at['2022-07-29', 'AU00Y06'] / 100

# Estimate annualized volatility
from statistics import stdev
cba_logret = np.log(stockprice['CBA']).diff().tail(-1)     # Daily log-returns
vol_est = stdev(cba_logret) / ((1 / 360) ** 0.5)

In [58]:
# Compute option price
op_price = BSprice(spot, strike, maturity, r, 0, vol_est, 'call')
op_price

11.006357816440243

A linear mapping of the changes in the call option price with respect to the underlying stock price and the risk-free rate is given by 

$$\Delta c = \frac{\partial c}{\partial s} \Delta s + \frac{\partial c}{\partial r} \Delta r = s \frac{\partial c}{\partial s} \frac{\Delta s}{s} + r\frac{\partial c}{\partial r} \frac{\Delta r}{r}.$$

The partial derivative $\partial c / \partial s$ can be computed using the `BSdelta()` function defined above. However, $\partial c / \partial r$, which is the *rho* of the call option, must be calculated using the formula

$$\frac{\partial c}{\partial r} = K T e^{-rT} \Phi(d_2(0,s)), \qquad \text{where } d_2(t,s) = \frac{\ln(s/K) + (r + \frac{1}{2} \sigma^2) (T-t)}{\sigma \sqrt{T-t}} - \sigma\sqrt{T-t},$$

and $\Phi(\cdot)$ is the cdf of the standard normal distribution (see page 11 of Week 2 Workshop notes). For this particular exercise, the rho can be calculated manually or a function can be defined to do so; we choose to define a function in case this calculation is needed in the future.

*REMARK:* For a put option, the rho is given by $\frac{\partial p}{\partial r} = -K T e^{-rT} \Phi(-d_2(0,s)).$

In [59]:
# Function to calculate option rho under the Black-Scholes-Merton model
def BSrho(spot_price, strike_price, time_to_maturity, risk_free_rate, dividend_yield, volatility, option_type):
    
    # Use mathematical notation for function inputs
    s = spot_price
    K = strike_price
    tau = time_to_maturity # (T-t) in notes
    r = risk_free_rate
    q = dividend_yield
    sigma = volatility
    
    # Calculate d1 and d2
    d1 = (np.log(s / K) + (r - q + 0.5 * sigma ** 2) * tau) / (sigma * np.sqrt(tau))
    d2 = d1 - sigma * np.sqrt(tau)
    
    # Calculate option delta
    if option_type == 'call':
        value = K * tau * np.exp(-r * tau) * sp.stats.norm.cdf(d2, 0.0, 1.0)
    
    if option_type == 'put':
        value = -K * tau * np.exp(-r * tau) * sp.stats.norm.cdf(-d2, 0.0, 1.0)
    
    return value

We first calculate the exposure with respect to each risk factor

In [60]:
# Calculate required Greeks
op_delta = BSdelta(spot, strike, maturity, r, 0, vol_est, 'call')
op_rho = BSrho(spot, strike, maturity, r, 0, vol_est, 'call')

# Calculate exposures
exposure = np.array([spot * op_delta, r * op_rho])
exposure

array([69.28393673,  0.74595301])

Finally, we construct the time series of returns in the underlying stock price and the zero rate and calculate the covariance matrix from there.

In [62]:
# Construct time series of risk factors
risk_factors = pd.DataFrame({'StockPrice' : stockprice['CBA'], 'RiskFreeRate' : auszero['AU00Y06']})

# Construct one-day percentage returns of the risk factors
risk_factors_ret = risk_factors.pct_change().tail(-1)

# Determine covariance and correlation matrices
risk_factors_cov = risk_factors_ret.cov()
risk_factors_corr = risk_factors_ret.corr()

print(risk_factors_cov)
print(risk_factors_corr)

              StockPrice  RiskFreeRate
StockPrice       0.00018     -0.000010
RiskFreeRate    -0.00001      0.004574
              StockPrice  RiskFreeRate
StockPrice      1.000000     -0.010966
RiskFreeRate   -0.010966      1.000000


A more popular risk factor mapping for European call options maps changes in the option price to the return and squared return of the underlying stock price. This mapping is referred to as the *delta-gamma* approximation and is given by

$$\Delta c = s \frac{\partial c}{\partial s} \frac{\Delta s}{s} + \frac{1}{2} s^2 \frac{\partial^2 c}{\partial s^2} \left(\frac{\Delta s}{s}\right)^2.$$

To establish this mapping, we need the option gamma (which can be calculated using the `BSgamma()` function defined above). The exposure to each risk factor can consequently be calculated.

In [64]:
# Calculate the option gamma
op_gamma = BSgamma(spot, strike, maturity, r, 0, vol_est)

# Calculate exposures
exposure = np.array([spot * op_delta, 0.5 * (spot ** 2) * op_gamma])
exposure

array([69.28393673, 99.16059244])

Finally, we construct the time series of returns and squared returns of the underlying stock price and calculate the resulting covariance matrix.

In [67]:
# Construct time series of stock prices and one-day percentage returns
risk_factors2 = pd.DataFrame({'StockPrice' : stockprice['CBA']})
risk_factors2_ret = risk_factors2.pct_change().tail(-1)

# Compute the squared percentage returns
risk_factors2_ret['SqRet'] = risk_factors2_ret ** 2

# Determine covariance and correlation matrices
risk_factors2_cov = risk_factors2_ret.cov()
risk_factors2_corr = risk_factors2_ret.corr()

print(risk_factors2_cov)
print(risk_factors2_corr)

              StockPrice         SqRet
StockPrice  1.796421e-04  3.137405e-07
SqRet       3.137405e-07  4.445317e-07
            StockPrice     SqRet
StockPrice    1.000000  0.035109
SqRet         0.035109  1.000000
